## **Installing and Testing YOLO-NAS**

In [121]:
!pip install super-gradients

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.2/964.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 512.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 

In [1]:
# Need to resize before input
from PIL import Image
im = Image.open("/content/photo.JPG")
newsize = (640, 640)
im1 = im.resize(newsize)

In [ ]:
import super_gradients

yolo_nas = super_gradients.training.models.get("yolo_nas_s", pretrained_weights="coco")#.cuda() #No CUDA for ONNX!
yolo_nas.predict(im1).show()

# **Converting to ONNX**

Two ways to get it in C++ (we will go with #2):

1) https://pytorch.org/tutorials/advanced/cpp_export.html
Export to TorchScript and Load with C++ API LibTorch

2) Export to ONYX and use ONNX RunTime (TensorRT is GPU Only). See Lei Mao's ONNX Runtime C++ Inference

In [ ]:
#Need to restart runtime between each run
from super_gradients.training import models
from super_gradients.common.object_names import Models

netS = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")
models.convert_to_onnx(model=netS, input_shape=(3,640,640), out_path="yolo_nas_s.onnx")

#netM = models.get(Models.YOLO_NAS_M, pretrained_weights="coco")
#models.convert_to_onnx(model=netM, input_shape=(3,640,640), out_path="yolo_nas_m.onnx")

#netL = models.get(Models.YOLO_NAS_L, pretrained_weights="coco")
#models.convert_to_onnx(model=netL, input_shape=(3,640,640), out_path="yolo_nas_l.onnx")

[2023-05-18 18:09:11] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-05-18 18:09:14] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-05-18 18:09:14] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-05-18 18:09:14] WARNING - export.py - Failed to import pytorch_quantization
[2023-05-18 18:09:14] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-05-18 18:09:15] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-05-18 18:09:15] WARNING - conversion.py - input_shape is deprecated and will be removed in the next major release. Use the input_size 

'yolo_nas_l.onnx'

# **Validating ONNX Model**

Checking output of ONNX model

In [1]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00


In [4]:
### IMAGE PREP
from PIL import Image
import torchvision.transforms as transforms

img = Image.open("/content/photo.JPG")

resize = transforms.Resize([640, 640])
img = resize(img)

img = img.convert('RGB')

to_tensor = transforms.ToTensor()
img = to_tensor(img)
img.unsqueeze_(0)

tensor([[[[0.5686, 0.5686, 0.5765,  ..., 0.2941, 0.2824, 0.2824],
          [0.5686, 0.5686, 0.5686,  ..., 0.2941, 0.2863, 0.2863],
          [0.5686, 0.5686, 0.5686,  ..., 0.2902, 0.2902, 0.2980],
          ...,
          [0.5373, 0.5765, 0.5176,  ..., 0.1176, 0.1137, 0.1137],
          [0.5020, 0.4980, 0.5961,  ..., 0.1725, 0.1412, 0.1333],
          [0.5569, 0.5765, 0.6353,  ..., 0.3059, 0.3333, 0.2902]],

         [[0.7490, 0.7490, 0.7569,  ..., 0.5020, 0.4941, 0.4941],
          [0.7490, 0.7490, 0.7529,  ..., 0.4980, 0.4980, 0.4941],
          [0.7490, 0.7490, 0.7490,  ..., 0.5020, 0.4980, 0.4941],
          ...,
          [0.5098, 0.5529, 0.5098,  ..., 0.1255, 0.1216, 0.1255],
          [0.4824, 0.4784, 0.5843,  ..., 0.1843, 0.1608, 0.1529],
          [0.5451, 0.5608, 0.6157,  ..., 0.3098, 0.3569, 0.3216]],

         [[0.9412, 0.9412, 0.9490,  ..., 0.7608, 0.7647, 0.7686],
          [0.9412, 0.9412, 0.9451,  ..., 0.7608, 0.7647, 0.7647],
          [0.9412, 0.9412, 0.9451,  ..., 0

In [5]:
### Running YOLO_NAS ONNX code to get outputs
#ort_outs[0] is bounding box coords (1,8400, 4)
#ort_outs[1] is one prediction confidences of all classes for all 8400 sections (1, 8400, 80)
import onnxruntime

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_session = onnxruntime.InferenceSession("/content/yolo_nas_s.onnx")
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img)}
ort_outs = ort_session.run(None, ort_inputs)

In [6]:
print(ort_outs[0].shape)
print(ort_outs[1].shape)

In [7]:
objects = [] 

for i in range(0,8400):
  if max(ort_outs[1][0][i]) >= 0.5: #if greater than 50% confidence
    objects.append(i) 

In [17]:
from PIL import Image
from numpy import asarray
im = Image.open("/content/photo.JPG")

resize = transforms.Resize([640, 640])
im = resize(im)

im = im.convert('RGB')

numpyimg = asarray(im)

In [11]:
import numpy as np
from typing import Tuple
def draw_text_box(
    image: np.ndarray,
    text: str,
    x: int,
    y: int,
    font: int,
    font_size: float,
    background_color: Tuple[int, int, int],
    thickness: int = 1,
) -> np.ndarray:
    """Draw a text inside a box
    :param image:               The image on which to draw the text box.
    :param text:                The text to display in the text box.
    :param x:                   The x-coordinate of the top-left corner of the text box.
    :param y:                   The y-coordinate of the top-left corner of the text box.
    :param font:                The font to use for the text.
    :param font_size:           The size of the font to use.
    :param background_color:    The color of the text box and text as a tuple of three integers representing RGB values.
    :param thickness:           The thickness of the text.
    :return: Image with the text inside the box.
    """
    text_color = (255,255,255) #best_text_color(background_color)
    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_size, thickness)
    text_left_offset = 7

    image = cv2.rectangle(image, (x, y), (x + text_width + text_left_offset, y - text_height - int(15 * font_size)), background_color, -1)
    image = cv2.putText(image, text, (x + text_left_offset, y - int(10 * font_size)), font, font_size, text_color, thickness, lineType=cv2.LINE_AA)
    return image


def draw_bbox(
    image: np.ndarray,
    title: str,
    color: Tuple[int, int, int],
    box_thickness: int,
    x1: int,
    y1: int,
    x2: int,
    y2: int,
) -> np.ndarray:
    """Draw a bounding box on an image.
    :param image:           Image on which to draw the bounding box.
    :param color:           RGB values of the color of the bounding box.
    :param title:           Title to display inside the bounding box.
    :param box_thickness:   Thickness of the bounding box border.
    :param x1:              x-coordinate of the top-left corner of the bounding box.
    :param y1:              y-coordinate of the top-left corner of the bounding box.
    :param x2:              x-coordinate of the bottom-right corner of the bounding box.
    :param y2:              y-coordinate of the bottom-right corner of the bounding box.
    """

    overlay = image.copy()
    overlay = cv2.rectangle(overlay, (x1, y1), (x2, y2), color, box_thickness)

    # Adapt font size to image shape.
    # This is required because small images require small font size, but this makes the title look bad,
    # so when possible we increase the font size to a more appropriate value.
    font_size = 0.25 + 0.07 * min(overlay.shape[:2]) / 100
    font_size = max(font_size, 0.5)  # Set min font_size to 0.5
    font_size = min(font_size, 0.8)  # Set max font_size to 0.8

    overlay = draw_text_box(image=overlay, text=title, x=x1, y=y1, font=2, font_size=font_size, background_color=color, thickness=1)

    return cv2.addWeighted(overlay, 0.75, image, 0.25, 0)

In [18]:
#Lists all possible bounding boxes, including multiple overlaps
boxes = []
for x in objects:
  boxes.append((int(ort_outs[0][0][x][0]), int(ort_outs[0][0][x][1]), int(ort_outs[0][0][x][2]), int(ort_outs[0][0][x][3])))
boxes = np.vstack(boxes)
print(boxes)

[[424 282 455 295]
 [424 282 460 295]
 [  0 287  20 313]
 [  0 288  20 313]
 [424 282 456 296]
 [424 282 456 296]
 [424 282 456 296]
 [  0 288  20 313]
 [  0 288  21 313]
 [ 18 291  37 305]
 [ 19 291  47 305]
 [ 18 291  58 305]
 [ 12 290  68 305]
 [  0 288  20 313]
 [  0 288  20 313]
 [304 296 343 353]
 [304 296 343 354]
 [304 296 343 353]
 [303 296 343 353]
 [303 296 344 355]
 [303 296 342 353]
 [303 296 342 354]
 [303 296 343 355]
 [303 295 340 353]
 [303 296 341 353]
 [  0 289  20 313]
 [ 20 291  36 305]
 [ 19 290  65 305]
 [  0 288  20 314]
 [145 276 204 536]
 [145 276 204 536]
 [195 278 244 535]
 [145 275 204 535]
 [144 275 203 535]
 [196 279 245 534]
 [196 279 245 534]
 [145 275 204 534]
 [144 275 204 535]
 [196 278 245 534]
 [196 278 245 534]
 [145 275 204 534]
 [196 279 245 534]
 [196 279 245 534]
 [144 276 205 534]
 [196 280 244 534]
 [196 279 244 534]
 [ 17  38 588 553]
 [ 14  39 584 549]
 [ 18  41 575 558]
 [ 22  32 635 546]
 [ 20  30 576 542]
 [ 11  33 578 542]
 [ 14  35 57

In [19]:
import cv2
img_w_bbox = numpyimg
i=0
for x in boxes:
  i = i+50
  img_w_bbox = draw_bbox(img_w_bbox, 'test_title',[i +25,i+10,i],1, x[0], x[1], x[2], x[3])

In [21]:
img = Image.fromarray(img_w_bbox, 'RGB')
img.show()

In [116]:
# Non-Maximum Suppression to consolidate overlapping bounding boxes
# Simple implementation, better to use torchvision.ops.boxes.batched_nms()
# I believe we need an algorithm that takes into account confidence and class
import numpy as np
# Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# if the bounding boxes integers, convert them to floats --
	# this is important since we'll be doing a bunch of divisions
	if boxes.dtype.kind == "i":
		boxes = boxes.astype("float")
	# initialize the list of picked indexes	
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list and add the
		# index value to the list of picked indexes
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		# find the largest (x, y) coordinates for the start of
		# the bounding box and the smallest (x, y) coordinates
		# for the end of the bounding box
		xx1 = np.maximum(x1[i], x1[idxs[:last]])
		yy1 = np.maximum(y1[i], y1[idxs[:last]])
		xx2 = np.minimum(x2[i], x2[idxs[:last]])
		yy2 = np.minimum(y2[i], y2[idxs[:last]])
		# compute the width and height of the bounding box
		w = np.maximum(0, xx2 - xx1 + 1)
		h = np.maximum(0, yy2 - yy1 + 1)
		# compute the ratio of overlap
		overlap = (w * h) / area[idxs[:last]]
		# delete all indexes from the index list that have
		idxs = np.delete(idxs, np.concatenate(([last],
			np.where(overlap > overlapThresh)[0])))
	# return only the bounding boxes that were picked using the
	# integer data type
	return boxes[pick].astype("int")

In [118]:
NMS_boxes = non_max_suppression_fast(boxes, 0.7)
print(NMS_boxes)

[[ 17  52 574 567]]


In [119]:
img_w_bbox = numpyimg
i=0
for x in NMS_boxes:
  i = i+50
  img_w_bbox = draw_bbox(img_w_bbox, 'test_title',[i +25,i+10,i],1, x[0], x[1], x[2], x[3])

In [22]:
img = Image.fromarray(img_w_bbox, 'RGB')
img.show()

# **Now we have our ONNX Model, pre and post processing all set.**  
# **And the rest is done in C++**

# **Some References Below**

See Lei Mao's ONNX Runtime C++ Inference  
https://github.com/leimao/ONNX-Runtime-Inference  
https://medium.com/@freshtechyy/deploying-pytorch-model-into-a-c-application-using-onnx-runtime-f9967406564b  
https://onnxruntime.ai/docs/tutorials/mnist_cpp.html  


https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html  
https://github.com/ultralytics/yolov5/issues/10159  
https://github.com/ultralytics/yolov5/blob/master/export.py  
https://pytorch.org/docs/stable/onnx.html



In [ ]:
batch_size=1
torch_model = yolo_nas
# Input to the model
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})